In [55]:
import requests
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [13]:
yinghui_csv_path = "../raw_data/long_sypnosis/title_only_df.csv"
df = pd.read_csv(yinghui_csv_path).drop(columns=['Unnamed: 0']).drop_duplicates().reset_index(drop=True)
imdb_id_ls = df["imdb_id"]

In [60]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36', 'Accept-Language': 'en-US,en;q=0.8'}
summary_full_ls = []
for i in tqdm(range(len(imdb_id_ls))):

    url = f'https://www.imdb.com/title/{imdb_id_ls[i]}/plotsummary/'
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
           warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    page_html = BeautifulSoup(response.text, 'html.parser')
    summaries_ls = page_html.find_all('div', class_='ipc-html-content-inner-div')
    summary_str = " ".join([summary.get_text() for summary in summaries_ls])
    summary_full_ls.append([imdb_id_ls[i], summary_str])

100%|██████████| 4624/4624 [1:43:40<00:00,  1.35s/it]


In [63]:
sypnosis_df = pd.DataFrame(summary_full_ls, columns=['imdb_id', 'sypnosis'])
sypnosis_df

,imdb_id,sypnosis
0,tt0259534,An anime adaptation of the Hindu epic the Rama...
1,tt0468569,When the menace known as the Joker wreaks havo...
2,tt0167260,Gandalf and Aragorn lead the World of Men agai...
3,tt9362722,"Miles Morales catapults across the Multiverse,..."
4,tt0075143,"The story of prophet ""Muhammad"" and the delive..."
...,...,...
4619,tt0109198,Four prostitutes join together to travel the O...
4620,tt0276830,"In 1870, after a brutal run-in with an outlaw ..."
4621,tt0120891,The two best special agents in the Wild West m...
4622,tt2479478,An outlaw who was raised by Native Americans d...


In [65]:
result_df = pd.merge(df, sypnosis_df)

In [66]:
result_df.to_csv('../raw_data/long_sypnosis/long_sypnosis.csv')